In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
"""
    Parameters
    ----------
    problem_formulation_id : int {0, ..., 5}
                             problem formulations differ with respect to the objectives
                             0: Total cost, and casualties
                             1: Expected damages, costs, and casualties
                             2: expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
                             3: costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
                             4: Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
                             5: disaggregate over time and space

    Notes
    -----
    problem formulations 4 and 5 rely on ArrayOutcomes and thus cannot straightforwardly
    be used in optimizations

    """
problem_formulation = 3
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [8]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2, policies=4)

[MainProcess/INFO] performing 2 scenarios * 4 policies * 1 model(s) = 8 experiments
100%|████████████████████████████████████████████| 8/8 [00:01<00:00,  4.77it/s]
[MainProcess/INFO] experiments finished


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,94,187.782210,10.0,0.572522,155.136383,10.0,0.562633,103.553160,1.0,0.666853,...,9,7,9,4,4,6,1,4,0,dikesnet
1,42,239.859325,1.0,0.192651,342.646866,1.0,0.209769,265.849692,1.5,0.140473,...,9,7,9,4,4,6,1,5,0,dikesnet
2,94,187.782210,10.0,0.572522,155.136383,10.0,0.562633,103.553160,1.0,0.666853,...,0,10,5,0,9,5,1,4,1,dikesnet
3,42,239.859325,1.0,0.192651,342.646866,1.0,0.209769,265.849692,1.5,0.140473,...,0,10,5,0,9,5,1,5,1,dikesnet
4,94,187.782210,10.0,0.572522,155.136383,10.0,0.562633,103.553160,1.0,0.666853,...,6,1,5,5,1,9,4,4,2,dikesnet
5,42,239.859325,1.0,0.192651,342.646866,1.0,0.209769,265.849692,1.5,0.140473,...,6,1,5,5,1,9,4,5,2,dikesnet
6,94,187.782210,10.0,0.572522,155.136383,10.0,0.562633,103.553160,1.0,0.666853,...,3,4,1,8,6,0,3,4,3,dikesnet
7,42,239.859325,1.0,0.192651,342.646866,1.0,0.209769,265.849692,1.5,0.140473,...,3,4,1,8,6,0,3,5,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,1.172995e+08,0.0,3.754300e+08,0.000000,1.413379e+08,0.000000,6.850229e+07,0.000000,1.242493e+08,0.000000,9.282000e+08,0.000000
1,1.172995e+08,0.0,3.754300e+08,0.000000,1.420618e+08,0.000462,6.850229e+07,0.000000,1.380980e+08,0.004461,9.282000e+08,424.803055
2,1.389030e+08,0.0,1.168492e+08,0.002041,1.295770e+08,0.000000,3.587330e+07,0.000000,1.115665e+08,0.004320,8.274000e+08,559.984361
3,1.389030e+08,0.0,1.701904e+08,0.017972,1.295770e+08,0.000000,6.852086e+07,0.004602,1.148695e+08,0.003698,8.274000e+08,3100.624297
4,2.388205e+08,0.0,2.644127e+08,0.000000,1.032933e+08,0.000000,3.432285e+07,0.000000,1.312716e+08,0.000000,1.239600e+09,0.000000
5,2.388205e+08,0.0,2.644127e+08,0.000000,1.032933e+08,0.000000,3.432285e+07,0.000000,1.312716e+08,0.000000,1.239600e+09,0.000000
6,2.484108e+08,0.0,1.007594e+08,0.000000,5.658500e+07,0.001210,2.717367e+07,0.000000,1.011575e+08,0.000000,1.267200e+09,249.740408
7,2.484108e+08,0.0,1.132422e+08,0.001401,1.794467e+08,0.024379,3.418734e+07,0.000348,1.011575e+08,0.000000,1.267200e+09,7353.444177


In [29]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations
# policy 4 doe niks
'''
A1 = Doesburg Upstream
A2 = Cortenoever Upmidstream
A3 = Zutphen Midstream
A4 = Gorssel Downmidstream
A5 = Deventer Downstream
0_RFR = Project Olburgen
1_RFR = Project Havikerwaard
2_RFR = project Tichelbeekse
3_RFR = Project Welsummer
4_RFR = Obstakelsverwijderen
dan 0 of 1 of 2 is de timestep waarin het wordt geddaan
'''

n_policies = 100

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 5, "A.3_DikeIncrease 1": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )),
    #     Policy(
    #     "policy x",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1}
    #     )
    # ),
    
    # Policy(
    #     "policy 2",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 3",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 4",
    #     **dict(
    #         get_do_nothing_dict(),
            
    #     )
    # ),
]

In [30]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 20
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 16 workers
[MainProcess/INFO] performing 20 scenarios * 2 policies * 1 model(s) = 40 experiments
100%|██████████████████████████████████████████| 40/40 [00:01<00:00, 33.26it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [31]:
experiments, outcomes = results

In [32]:
# only works because we have scalar outcomes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
outcomes=pd.DataFrame(outcomes)

# aggregating outcomes
cost_keys = [k for k in outcomes if 'Total Costs' in k]
outcomes['Total costs'] = np.sum([outcomes[k] for k in cost_keys], axis=0)

cost_keys = [k for k in outcomes if 'Number of Deaths' in k]
outcomes['Number of Deaths'] = np.sum([outcomes[k] for k in cost_keys], axis=0)


In [33]:
outcomes

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,Total costs,Number of Deaths
0,0.000000e+00,0.00000,3.898607e+07,0.034366,6.286508e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,345900000.0,0.0,4.477511e+08,0.034366
1,0.000000e+00,0.00000,6.572357e+07,0.059133,6.286508e+07,0.000000,7.893091e+07,0.035352,0.000000e+00,0.000000,345900000.0,0.0,5.534196e+08,0.094485
2,0.000000e+00,0.00000,0.000000e+00,0.000000,6.286508e+07,0.000000,1.567773e+07,0.008131,0.000000e+00,0.000000,345900000.0,0.0,4.244428e+08,0.008131
3,0.000000e+00,0.00000,8.538116e+06,0.008554,6.286508e+07,0.000000,6.410231e+07,0.031397,2.331395e+07,0.021641,345900000.0,0.0,5.047194e+08,0.061592
4,2.850705e+09,1.69452,8.157398e+06,0.006742,6.286508e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,345900000.0,0.0,3.267628e+09,1.701262
5,0.000000e+00,0.00000,8.762661e+08,0.801788,6.286508e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,345900000.0,0.0,1.285031e+09,0.801788
6,0.000000e+00,0.00000,1.491283e+08,0.145511,6.286508e+07,0.000000,0.000000e+00,0.000000,3.434229e+08,0.331796,345900000.0,0.0,9.013162e+08,0.477308
7,0.000000e+00,0.00000,2.396128e+08,0.267604,6.286508e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,345900000.0,0.0,6.483778e+08,0.267604
8,0.000000e+00,0.00000,2.896453e+08,0.263028,6.286508e+07,0.000000,8.272481e+06,0.004100,7.228669e+08,0.657929,345900000.0,0.0,1.429550e+09,0.925057
9,0.000000e+00,0.00000,2.234798e+07,0.023109,6.286508e+07,0.000000,5.487910e+07,0.028643,4.359785e+08,0.419115,345900000.0,0.0,9.219706e+08,0.470867


In [20]:
experiments = pd.DataFrame(experiments)
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,EWS_DaysToThreat,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,67,321.558342,10.0,0.625956,267.910155,10.0,0.294151,111.988465,1.0,0.923138,172.897493,1.0,0.501047,333.270236,1.0,0.081570,2.5,4.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,26,policy 1,dikesnet
1,56,192.858338,10.0,0.897534,278.511540,10.0,0.975744,284.807613,10.0,0.350281,252.297898,1.0,0.691248,114.668973,1.5,0.524574,2.5,2.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,27,policy 1,dikesnet
2,111,171.332788,1.0,0.400832,97.627655,1.5,0.814578,97.241309,1.0,0.263546,65.929115,1.5,0.566724,307.984314,1.0,0.864465,1.5,1.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,28,policy 1,dikesnet
3,74,68.525368,1.5,0.906589,332.196035,10.0,0.773712,48.024367,1.5,0.588078,80.492196,10.0,0.828983,186.551136,10.0,0.227859,1.5,3.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,29,policy 1,dikesnet
4,3,187.324884,1.0,0.835988,121.254861,1.5,0.593747,236.636835,10.0,0.860397,174.153866,1.5,0.499209,137.690646,10.0,0.742810,1.5,4.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,30,policy 1,dikesnet
5,50,211.897417,10.0,0.544194,53.502469,10.0,0.344705,134.527900,1.0,0.997737,276.990621,1.5,0.067109,104.024838,10.0,0.835675,4.5,3.5,1.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,31,policy 1,dikesnet
6,62,273.739088,10.0,0.045817,234.286558,10.0,0.408340,39.860960,1.0,0.158331,111.285115,1.5,0.888283,76.344513,1.5,0.278858,3.5,2.5,2.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,32,policy 1,dikesnet
7,94,243.385281,1.0,0.588992,221.599659,10.0,0.875752,164.518236,1.5,0.214084,134.167176,1.0,0.997200,241.196521,1.0,0.798780,2.5,2.5,4.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,33,policy 1,dikesnet
8,85,308.910950,1.5,0.702300,153.562926,1.5,0.905171,262.833373,10.0,0.738317,322.078933,1.0,0.370436,39.681741,1.0,0.115522,2.5,1.5,1.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,34,policy 1,dikesnet
9,124,96.440493,1.5,0.334118,134.499104,1.5,0.722615,307.666956,1.0,0.404007,157.596619,1.0,0.233443,235.229063,1.5,0.987650,3.5,3.5,3.5,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,35,policy 1,dikesnet
